In [2]:
import requests
import json
import os
import time

from functools import wraps
from pathlib import Path
from enum import Enum

# Config

In [20]:
API_ROUTE = "https://api.twitter.com/2/"
STREAM_ROUTE = "tweets/search/stream"
RULES_ROUTE = "tweets/search/stream/rules"
ROOT_FOLDER = r"/home/tyra/Documents/Collectes"
MAX_SIZE = 1000000
TOKEN = ""
params = {
    "tweet.fields": "entities,public_metrics",
    "expansions": "author_id,in_reply_to_user_id,attachments.media_keys",
    "media.fields": "url",
    "user.fields": "id,verified"
}

#### Load Token

In [24]:
with open("/home/tyra/Documents/CERES/credentials.json", 'r') as f:
    TOKEN = f"Bearer {json.load(f)['token']}"

In [25]:
s = requests.Session()
s.headers.update({"Authorization": TOKEN})

In [6]:
class Media(Enum):
    PHOTO = 1
    VIDEO = 2
    GIF = 3
    
    def __eq__(self, other):
        if isinstance(other, str):
            return self.name.lower() == other

In [7]:
def timeit(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        res = f(*args, **kwargs)
        print(f"{f.__name__} executed in {time.time() - t1}")
        return res
    return wrapper

In [8]:
# test_handle_tweet
tweet = """
{
    "data": {
        "attachments": {
            "media_keys": [
                "3_1437787186561228800"
            ]
        },
        "author_id": "21313364",
        "entities": {
            "urls": [
                {
                    "start": 129,
                    "end": 152,
                    "url": "https://t.co/1kUyRMrCmS",
                    "expanded_url": "https://www.nouvelobs.com/droits-des-femmes/20210914.OBS48633/il-y-a-un-afflux-considerable-de-demandes-le-gouvernement-veut-reduire-les-delais-d-attente-pour-la-pma-d-un-an-a-six-mois.html?utm_term=Autofeed&utm_medium=Social&utm_source=Twitter#Echobox=1631630086",
                    "display_url": "nouvelobs.com/droits-des-fem…",
                    "images": [
                        {
                            "url": "https://pbs.twimg.com/news_img/1437787191103692800/Wg5T_O5D?format=jpg&name=orig",
                            "width": 1024,
                            "height": 512
                        },
                        {
                            "url": "https://pbs.twimg.com/news_img/1437787191103692800/Wg5T_O5D?format=jpg&name=150x150",
                            "width": 150,
                            "height": 150
                        }
                    ],
                    "status": 200,
                    "title": "« Il y a un afflux considérable de demandes » : le gouvernement veut réduire les délais d’attente pour la PMA d’un an à six mois",
                    "description": "Le Parlement a voté fin juin la procréation médicalement assistée pour toutes les femmes, qu’elles soient hétérosexuelles, homosexuelles ou monoparentales. Une enveloppe de 8 millions d’euros doit permettre de répondre à l’afflux de demandes.",
                    "unwound_url": "https://www.nouvelobs.com/droits-des-femmes/20210914.OBS48633/il-y-a-un-afflux-considerable-de-demandes-le-gouvernement-veut-reduire-les-delais-d-attente-pour-la-pma-d-un-an-a-six-mois.html?utm_term=Autofeed&utm_medium=Social&utm_source=Twitter#Echobox=1631630086"
                },
                {
                    "start": 153,
                    "end": 176,
                    "url": "https://t.co/1uGLGEP38c",
                    "expanded_url": "https://twitter.com/lobs/status/1437787188868091906/photo/1",
                    "display_url": "pic.twitter.com/1uGLGEP38c"
                }
            ]
        },
        "id": "1437787188868091906",
        "organic_metrics": {},
        "text": "« Il y a un afflux considérable de demandes » : le gouvernement veut réduire les délais d’attente pour la PMA d’un an à six mois https://t.co/1kUyRMrCmS https://t.co/1uGLGEP38c"
    },
    "includes": {
        "media": [
            {
                "media_key": "3_1437787186561228800",
                "type": "photo",
                "url": "https://pbs.twimg.com/media/E_QL1BWXIAAEEPV.jpg"
            }
        ],
        "users": [
            {
                "id": "21313364",
                "name": "L'Obs",
                "username": "lobs"
            }
        ]
    },
    "matching_rules": [
        {
            "id": 1437707103754506241,
            "tag": "PMA"
        }
    ]
}"""
handle_tweet(json.loads(tweet))


NameError: name 'handle_tweet' is not defined

In [11]:
def handle_media(tweet):
    # check if there are some media in the tweet
    if not tweet.get('includes', {}).get('media', False):
        return
    # extract the media
    media = tweet['includes']['media']
    for medium in media:
        if medium['type'] == Media.PHOTO:
            handle_photo(**medium)
        else:
            print(f"unhandled media type currently: {medium['type']}")

In [10]:
def handle_photo(media_key=None, url=None, **kwargs):
    """
    save a picture to disk
    pictures are saves in root_directory/images and unlike tweet are not saved by tags
    # TODO: add metadata + sha1 + occurences count
    # TODO: split the download part to use it for every media
    """
    if not media_key or not url:
        raise ValueError("Missing field when trying to save image")
    file_type = url.split('.')[-1]
    file_name = f"{media_key}.{file_type}"
    try:
        res = requests.get(url)
        print("downloading image")
    except requests.RequestException:
        raise ValueError(f"There was an error when downloading the image with following url: {url}, please check your connection or url")
    # save to disk
    # TODO: maybe ensure the creation of '/images' elsewhere ? 
    directory = Path(os.path.join(ROOT_FOLDER, 'images'))
    directory.mkdir(parents=True, exist_ok=True)
    with open(os.path.join(directory, file_name), 'wb') as f:
        f.write(res.content)

In [9]:
@timeit
def handle_tweet(tweet):
    id = tweet['data']['id']
    
    # filter tags:
    tags = list(set([r['tag'] for r in tweet["matching_rules"]]))
    
    # save media:
    handle_media(tweet)
    
    # save collected tweet in every tag
    for tag in tags:
        # create directory if not exist
        directory = Path(os.path.join(ROOT_FOLDER), tag)
        directory.mkdir(parents=True, exist_ok=True)
        with open(os.path.join(directory, f"{id}.json"), 'w', encoding='utf-8') as f:
            json.dump(tweet, f, indent=4, ensure_ascii=False)

In [12]:
def get_folder_size(path):
    return sum(f.stat().st_size for f in Path(path).glob('**/*') if f.is_file())

In [13]:
@timeit
def has_free_space():
    if get_folder_size(ROOT_FOLDER) < MAX_SIZE:
        return True
    raise OSError("The maxsize of the storage directory has been reached")

In [29]:
def get_rules():
    """
    Allow to get all rules currently active for the collect
    """
    rules = s.get(API_ROUTE + RULES_ROUTE)
    return rules.json()

In [33]:
def get_tags_from_rules(rules):
    return list(set([r['tag'] for r in rules['data']]))

In [15]:
def init_storages(folders=[]):
    """
    Ensure folders are properly created at the begining of the collect
    """
    for f in folders:
        directory = Path(os.path.join(ROOT_FOLDER, f))
        directory.mkdir(parents=True, exist_ok=True)

In [ ]:
def collect():
    # first ensure all folders needed are created
    init_storages(['users', 'images', 'videos', 'gifs'])
    with s.get(url, params=params, stream=True, timeout=5000) as resp:
        if resp.status_code != 200:
            print(f"error {resp.status_code}")
            print(resp.content)
        for line in resp.iter_lines():
            if line and has_free_space():
                print(line.decode("utf-8"))
                handle_tweet(json.loads(line.decode("utf-8")))
            else:
                print("waiting for new tweets")